# TP2.2 : Stratégies d’investissement (bis)
Notebook fait par Alexandre Boistard, Ethan Trentin.

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
import seaborn as sns
from tqdm import tqdm

In [2]:
# Question 1. Nous allons d'abord implémenter des stratégies du tp 2.1
def ema_crossover_strategy(df, tickers, short_window, long_window):
    signals = pd.DataFrame(index=df.index)
    for stock in tickers:
        # Compute the short and long EMAs. Change half_life to span if desired.
        signals[stock + " Short EMA"] = df[stock].ewm(halflife =short_window).mean()
        signals[stock + " Long EMA"] = df[stock].ewm(halflife =long_window).mean()
        # Generate signals: 1 if short EMA > long EMA, else -1.
        signals[stock + " Signal"] = np.where(
            signals[stock + " Short EMA"] > signals[stock + " Long EMA"],
            1,
            -1
        )
    return signals

In [3]:
def mean_revert_strategy(df, tickers, time_window, k):
    signals = pd.DataFrame(index=df.index)
    for stock in tickers:
        # Compute the MA.
        signals[stock + " MA"] = df[stock].rolling(window=time_window).mean()
        # Generate signals: -1 if > k*simga, 0 between, else 1.
        signals[stock + " Signal"] = np.where(
            df[stock] > signals[stock + " MA"] + k * df[stock].rolling(window=time_window).std(),
            -1,
            np.where(
                df[stock] < signals[stock + " MA"] - k * df[stock].rolling(window=time_window).std(),
                1,
                0
            )
        )
    return signals